<a href="https://colab.research.google.com/github/gshreya5/colab/blob/main/scrape_mtsample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SCRAPE MTSAMPLE

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import json
import pandas as pd

In [ ]:
url = 'https://www.mtsamples.com/'
html = requests.get(url)
print(f'status : {html.status_code}')

soup = BeautifulSoup(html.text)

status : 200


In [ ]:
links = soup.find('ol').find_all('a')

In [ ]:
def add_data_to_datasets(sample_soup):
      record = {}

      col_names = ["Medical Specialty","Sample Name","Description","TRANSCIPTION","Keywords"]
      data = re.sub("\(.*?\)|\[.*?\]","",sample_soup).replace('.push;','')

      record["Medical Specialty"] = l.text
      record["Sample Name"] = sample.text
      record["Description"],record["TRANSCIPTION"] = data.split("Description:",1)[1].split(":",1)
      last_word = record["Description"].split()[-1]
      record["Description"] = ' '.join(record["Description"].split()[:-1])
      record["TRANSCIPTION"] = last_word + ":" + record["TRANSCIPTION"].split('See More')[0]
      record["Keywords"] = data.split("Keywords:",1)[1]

      datasets.append(json.loads(json.dumps(record,indent=4)))

In [ ]:
datasets = []
bad_links = []

for l in links:
  print(f'Medical Specialty : {l.text}')
  link_url = l.get('href')
  link_soup = BeautifulSoup(requests.get(url+link_url).text)
  samples = link_soup.find('table').find_all('a')
  for sample in samples:
    print(f'Sample Name : {sample.text}')
    #time.sleep(4)
    sample_url = sample.get('href')
    sample_soup=""
    
    try :
      sample_soup = BeautifulSoup(requests.get(url+sample_url).text).find(class_ = 'hilightBold').text
      add_data_to_datasets(sample_soup)
    except :
      try:
        sample_soup = BeautifulSoup(requests.get(url+sample_url).text).find(class_ = 'hilightBold').get_text()
        add_data_to_datasets(sample_soup)
      except:
        print(url+sample_url)
        bad_links.append(url+sample_url)

In [ ]:
bad_links

In [ ]:
df = pd.DataFrame.from_dict(datasets)
df

""
